### Import libraries

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers

### Load and split data

In [2]:
ds_train = keras.utils.image_dataset_from_directory(
    directory = "../dataset/",
    labels='inferred',
    label_mode='categorical',
    batch_size=64,
    image_size=(100, 100),
    shuffle=True,
    seed=0,
    validation_split=0.2,
    subset="training"
)

ds_val = keras.utils.image_dataset_from_directory(
    directory = "../dataset/",
    labels='inferred',
    label_mode='categorical',
    batch_size=64,
    image_size=(100, 100),
    shuffle=True,
    seed=0,
    validation_split=0.2,
    subset="validation"
)

ds_test = ds_val.take(5)
ds_val = ds_val.skip(5)

Found 7750 files belonging to 14 classes.
Using 6200 files for training.


2022-04-26 19:31:36.774370: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-26 19:31:36.822490: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-26 19:31:36.822687: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-26 19:31:36.823275: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Found 7750 files belonging to 14 classes.
Using 1550 files for validation.


In [3]:
print('Batches for testing -->', ds_test.cardinality())
print('Batches for validating -->', ds_val.cardinality())

Batches for testing --> tf.Tensor(5, shape=(), dtype=int64)
Batches for validating --> tf.Tensor(20, shape=(), dtype=int64)


### Create model

In [4]:

img_height = 100
img_width = 100
num_classes = 14
img_channels = 3

model = keras.Sequential([
  layers.Conv2D(16, 3, padding='same', activation='relu',input_shape=(img_height, img_width, img_channels)),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.2),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes, activation='softmax')
])



### Compile model

In [5]:
model.compile(optimizer="Adam", loss="categorical_crossentropy", metrics=["accuracy"])

### Train model

In [7]:
model.fit(ds_train, epochs=42, validation_data=ds_val)

Epoch 1/42
97/97 [==============================] - 1s 13ms/step - loss: 0.0377 - accuracy: 0.9895 - val_loss: 0.3675 - val_accuracy: 0.9187
Epoch 2/42
97/97 [==============================] - 1s 13ms/step - loss: 0.0363 - accuracy: 0.9890 - val_loss: 0.4591 - val_accuracy: 0.9033
Epoch 3/42
97/97 [==============================] - 1s 13ms/step - loss: 0.0291 - accuracy: 0.9918 - val_loss: 0.4567 - val_accuracy: 0.9171
Epoch 4/42
97/97 [==============================] - 1s 13ms/step - loss: 0.0341 - accuracy: 0.9905 - val_loss: 0.5441 - val_accuracy: 0.9089
Epoch 5/42
97/97 [==============================] - 1s 13ms/step - loss: 0.0317 - accuracy: 0.9906 - val_loss: 0.4812 - val_accuracy: 0.9171
Epoch 6/42
97/97 [==============================] - 1s 13ms/step - loss: 0.0283 - accuracy: 0.9903 - val_loss: 0.4317 - val_accuracy: 0.9187
Epoch 7/42
97/97 [==============================] - 1s 13ms/step - loss: 0.0478 - accuracy: 0.9850 - val_loss: 0.5845 - val_accuracy: 0.8992
Epoch 8/42
97

### Evaluate model on test set

In [8]:
model.evaluate(ds_test)

5/5 [==============================] - 0s 9ms/step - loss: 0.7457 - accuracy: 0.9125


[0.7456718683242798, 0.9125000238418579]

### Save model

In [9]:
model.save("../model3")

2022-04-26 19:33:31.139152: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: ../model3/assets
